#### Import Libraries

In [2]:
import os
import selenium
import pandas as pd
import time
from selenium import webdriver
import io
import requests as r
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.common.exceptions import ElementClickInterceptedException
from bs4 import BeautifulSoup
from time import sleep
from config import APP_BASE_URL,EMAIL,PASSWORD, CATEGORY_1, CATEGORY_2

In [ ]:
#Install Driver
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

In [27]:
def login_and_get_table(EMAIL, PASSWORD, driver):
    # driver.close()
    driver.get(APP_BASE_URL+'/users/sign_in')
    # find username/email field and send the username itself to the input field
    driver.find_element(By.ID,"user_email").send_keys(EMAIL)
    # find password input field and insert password as well
    driver.find_element(By.ID,"user_password").send_keys(PASSWORD)
    # click submit button which has tag "mt-5"
    driver.find_element(By.CLASS_NAME,"mt-5").click()
    sleep(2)
    print('Login Successful, Navigating To Inventory Page')
    # navigate to actual page to scrape data
    driver.get(APP_BASE_URL+'/setup_inventory')
    sleep(3)
    driver.find_element(By.CLASS_NAME,"ingredientsList").click()
    inventory_page = driver.page_source
    raw_page_html = BeautifulSoup(inventory_page   ,'lxml')

    table_headers = []
    for table_header in raw_page_html.find_all('th'):
        col_name = table_header.text
        table_headers.append(col_name)   #Append all headers found to a list. 
    table_headers= table_headers[:-1]    #excluding last column header which had button links
    return table_headers


def extract_and_transform(table_headers):

    Categs_to_Extract = [CATEGORY_1, CATEGORY_2]

    extracted_inventory = pd.DataFrame()
    for category in Categs_to_Extract:

        driver.find_element(By.CLASS_NAME,"filterByCategDropdown").click()
        sleep(2)
        driver.find_element(By.PARTIAL_LINK_TEXT, category).click()

        all_inventory_data = pd.DataFrame()
        next_page_exists = True
        page = 1
        while (next_page_exists):
            try:
                page_info = driver.page_source
                page_info_html =  BeautifulSoup(page_info,'lxml')
                table_data = []
                
                for table_row in page_info_html.find_all('tr'):
                    rows = {}
                    for cell_data, table_header in zip(table_row.find_all('td'),table_headers):
                        rows[table_header] = cell_data.text.replace('\n','').strip()
                    table_data.append(rows)
                page_data_extracted = pd.DataFrame(table_data)
                all_inventory_data = pd.concat([all_inventory_data,page_data_extracted])
                sleep(5)
                next_page = driver.find_element(By.XPATH,'//li[@class="page-item next"]/a').click()
                sleep(2)
                page+=1
                print('now fetching data from next page : {}'.format(page))
            except Exception as E:
                print('Seems there are no other pages: or ', type(E).__name__,'\n\n', 'Moving to next Category: {}.............'.format(Categs_to_Extract[1]))
                extracted_inventory = pd.concat([extracted_inventory,all_inventory_data])
                extracted_inventory.dropna(axis = 0, subset =['Name'],inplace=True)
                next_page_exists = False
                sleep(3)
                break
    return extracted_inventory

def show_previews_and_export(extracted_inventory):
    # excluded prints - propietary
    # print('Preview of Top 5 Rows:','\n','*---*'*10,'\n\n',extracted_inventory.head(10) ) 
    sleep(3)
    # print('Preview of Bottom 5 Rows:','\n','*---*'*10,'\n\n',extracted_inventory.tail(10) )
    sleep(3)
    print('Exporting {} rows of data to Csv'.format(len(extracted_inventory)))
    extracted_inventory.to_csv('extracted_inventory_data.csv',index=False)


def main():
  start_time = time.time()
  print('Now Running script')
  table_headers = login_and_get_table(EMAIL=EMAIL,PASSWORD= PASSWORD, driver=driver)
  print('Done Extracting Table_headers ..... Commencing Data Scraping',
            '\n','Done With Page 1 Extraction')
  extracted_inventory = extract_and_transform(table_headers)
  show_previews_and_export(extracted_inventory)
  driver.get(APP_BASE_URL+'/users/sign_out') #log out of app
  print("\nThis took %s seconds." % (time.time() - start_time))
  print('Done with Webscraping')
        


if __name__ == "__main__":
	main()
        

Now Running script
Login Successful, Navigating To Inventory Page
Done Extracting Table_headers ..... Commencing Data Scraping 
 Done With Page 1 Extraction
now fetching data from next page : 2
now fetching data from next page : 3
now fetching data from next page : 4
now fetching data from next page : 5
now fetching data from next page : 6
now fetching data from next page : 7
Seems there are no other pages: or  NoSuchElementException 

 Moving to next Category: SD Finished Product.............
now fetching data from next page : 2
now fetching data from next page : 3
now fetching data from next page : 4
now fetching data from next page : 5
now fetching data from next page : 6
Seems there are no other pages: or  NoSuchElementException 

 Moving to next Category: SD Finished Product.............
Exporting 231 rows of data to Csv

This took 129.1827654838562 seconds.
Done with Webscraping
